In [46]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
# !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
# !tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
# !pip install -q findspark

In [78]:
# Spark 8:39 1/5/2022
import findspark
findspark.init()
findspark.find()

'/content/spark-3.0.0-bin-hadoop3.2'

In [79]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
  .appName("Colab").config("spark.ui.port", "4050")\
  .getOrCreate()

In [80]:
spark

In [113]:
df = spark.read.csv("train.csv", header=True, inferSchema=True).toPandas()
df = df.sample(10000)
target = df['Purchase']
df = df.drop(["Purchase", "User_ID", "Product_ID"], axis=1)

In [114]:
from sklearn.preprocessing import LabelEncoder as le, StandardScaler as ss
le = le()
ss = ss()

In [115]:
for col in train.columns:
  if df[col].dtype != object:
    df[col].fillna(df[col].median(), inplace=True)
    df[col] = ss.fit_transform(df[col].values.reshape(-1,1))  
  else:
    df[col] = le.fit_transform(df[col].astype(str))

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

In [117]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2)

In [118]:
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()
stack = StackingRegressor(estimators=[("RFR", rfr), ("GBR", gbr)])

In [119]:
stack.fit(X_train, y_train)

StackingRegressor(estimators=[('RFR', RandomForestRegressor()),
                              ('GBR', GradientBoostingRegressor())])

In [120]:
train_score = stack.score(X_train, y_train)

In [121]:
print(f"Train Score: {train_score}")

Train Score: 0.6721576435228585


In [122]:
from sklearn.metrics import r2_score
predictions = stack.predict(X_test)

In [124]:
r2 = r2_score(y_test, predictions)

In [125]:
print(f"Test Score: {r2}")

Test Score: 0.6460476094135336
